<h1> Ray line to Image Line </h1>

First import the libraries:

In [1]:
%matplotlib inline

import scipy.io as spio
import scipy
import matplotlib.pyplot as plt
import numpy as np
import h5py
import imageio
import cv2
import keras
from keras.utils.vis_utils import plot_model
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from scipy.signal import hilbert
from scipy.fftpack import *

Using TensorFlow backend.
/home/hero/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hero/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hero/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hero/anaconda3/lib/python3.7/site-packages/tensorflow/py

Load the data:

In [2]:
size_train=1300
size_test=100
size_valid=80
length=384
length_rf=length
length_img=length
rf_place = 'RF_1smpl_fixedfocus/Comp16/'
img_place = 'Images_1smpl_fixedfocus/'
trainRF=np.empty([size_train*128,length])
trainIMG=np.empty([size_train*128,length])
testRF=np.empty([size_test*128,length])
testIMG=np.empty([size_test*128,length])
validRF=np.empty([size_valid*128,length])
validIMG=np.empty([size_valid*128,length])
pulse = spio.loadmat('pulse.mat', squeeze_me=True)
pulse = pulse['temp']
for i in range(size_train):
    trainRF_temp = spio.loadmat(rf_place+str(i+1)+'.mat', squeeze_me=True)
    trainRF_temp = trainRF_temp['compressed']

    for j in range(128):
        trainRF[i*128+j,:]=trainRF_temp[:,j]
    trainIMG_temp = spio.loadmat(img_place+str(i+1)+'.mat', squeeze_me=True)
    trainIMG_temp = trainIMG_temp['imgData']

    for j in range(128):
        trainIMG[i*128+j,:]=trainIMG_temp[:,j]
for i in range(size_test):
    testRF_temp = spio.loadmat(rf_place+str(i+size_train+1)+'.mat', squeeze_me=True)
    testRF_temp = testRF_temp['compressed']
    for j in range(128):
        testRF[i*128+j,:]=testRF_temp[:,j]
    testIMG_temp = spio.loadmat(img_place+str(i+size_train+1)+'.mat', squeeze_me=True)
    testIMG_temp = testIMG_temp['imgData']

    for j in range(128):
        testIMG[i*128+j,:]=testIMG_temp[:,j]
for i in range(size_valid):
    validRF_temp = spio.loadmat(rf_place+str(i+size_train+size_test+1)+'.mat', squeeze_me=True)
    validRF_temp = validRF_temp['compressed']
    for j in range(128):
        validRF[i*128+j,:]=validRF_temp[:,j]
    validIMG_temp = spio.loadmat(img_place+str(i+size_train+size_test+1)+'.mat', squeeze_me=True)
    validIMG_temp = validIMG_temp['imgData']

    for j in range(128):
        validIMG[i*128+j,:]=validIMG_temp[:,j]

Normalize the data. The RF data has been just scaled down, while the Image data gets fitted between 0 and 1. RF should not be fit between 0 and 1 because it is a wave signal and is expected to have negative values.

In [3]:
trfmin=trainRF.min()
timgmin=trainIMG.min()

trfmax=np.absolute(trainRF.max())
trainRF1=trainRF/trfmax
trainIMG=trainIMG-timgmin
timgmax=trainIMG.max()
trainIMG1=trainIMG/timgmax

testRF1=testRF/trfmax
testIMG=testIMG-timgmin
testIMG1=testIMG/timgmax

validRF1=validRF/trfmax
validIMG=validIMG-timgmin
validIMG1=validIMG/timgmax

Add a third dimension. This is needed because of the way that Conv1D layers work in keras.

In [5]:
trainRF1=trainRF1.reshape(size_train*128,length,1)
testRF1=testRF1.reshape(size_test*128,length,1)
validRF1=validRF1.reshape(size_valid*128,length,1)

Create a model.

In [ ]:
model = keras.Sequential()

model.add(Conv1D(12, 7, padding = 'same', input_shape=(length,1)))
model.add(LeakyReLU())

model.add(Conv1D(16, 7, padding = 'same'))
model.add(LeakyReLU())

model.add(Conv1D(24, 7, padding = 'same'))
model.add(LeakyReLU())

model.add(Conv1D(48, 7, padding = 'same'))  #optimal results with 48 filters
model.add(LeakyReLU())

model.add(Flatten())

model.add(Dense(length//2))
model.add(LeakyReLU())

model.add(Dense(length//2))
model.add(LeakyReLU())

model.add(Dense(length))

model.add(Activation('relu'))


model.summary()

In [ ]:
opt = keras.optimizers.Adam(lr=1e-5)

model.compile(optimizer = opt, loss = 'mean_squared_error')

Finally, train the model:

In [ ]:
history = model.fit(trainRF1, # Features
                      trainIMG1, # Target vector
                      epochs=2, # Number of epochs
                      verbose=1, # Print description after each epoch
                      batch_size=128, # Number of observations per batch
                      validation_data=(testRF1, testIMG1)) # Data for evaluation

score = model.evaluate(validRF1, validIMG1, batch_size=128)
print('The score is ', score)

Verification of the model. It is important to plot the resulting image as well.

In [ ]:
pp=model.predict(testRF1, verbose=1)

In [ ]:
pic_no=3
%pylab
plt.imshow(pp[pic_no*128:(pic_no+1)*128,:].T)#, cmap=plt.cm.get_cmap('gray'))
plt.figure()
plt.imshow(testIMG1[pic_no*128:(pic_no+1)*128,:].T)#, cmap=plt.cm.get_cmap('gray'))
print(np.mean(np.power(pp[pic_no*128:(pic_no+1)*128,:].T - trainIMG1[pic_no*128:(pic_no+1)*128,:].T, 2)))

Optionally, the model can be saved:

In [ ]:
model.save('rf2img_leakyrelu1')